In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [ ]:
#export
from exp.nb_01 import *

def get_data():
#
#
#
#

def normalize(x, m, s): #

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = #

In [ ]:
train_mean,train_std = #
train_mean,train_std

(tensor(3.0614e-05), tensor(1.))

In [ ]:
#export
def test_near_zero(a,tol=1e-3): #

In [ ]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [ ]:
n,m = #
c = #
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [ ]:
# num hidden
nh = 50

In [ ]:
# simplified kaiming init / he init
w1 = #
b1 = #
w2 = torch.randn(nh,1)#
b2 = torch.zeros(1)

In [ ]:
test_near_zero(w1.mean())
test_near_zero(w1.std()#

In [ ]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [ ]:
def lin(x, w, b): #

In [ ]:
t = lin(x_valid, w1, b1)

In [ ]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0600), tensor(0.9136))

In [ ]:
def relu(x): #

In [ ]:
t = relu(lin(x_valid, w1, b1))

In [ ]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3919), tensor(0.5513))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [ ]:
# kaiming init / he init for relu
w1 = #

In [ ]:
w1.mean(),w1.std()

(tensor(-4.4581e-05), tensor(0.0506))

In [ ]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5243), tensor(0.7824))

In [ ]:
#export
from ## import init

In [ ]:
w1 = torch.zeros(m,nh)
init.#
t = relu(lin(x_valid, w1, b1))

In [ ]:
w1.mean(),w1.std()

(tensor(0.0002), tensor(0.0505))

In [ ]:
t.mean(),t.std()

(tensor(0.6315), tensor(0.8479))

In [ ]:
w1.shape

torch.Size([784, 50])

In [ ]:
import torch.nn

In [ ]:
torch.nn.Linear(##).weight.shape

torch.Size([50, 784])

In [ ]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [ ]:
# what if...?
def relu(x): #

In [ ]:
# kaiming init / he init for relu
w1 = #
t1 = #
t1.mean(),t1.std()

(tensor(0.1596), tensor(0.8906))

In [ ]:
def model(xb):
    l1 = lin#
    l2 = relu#
    l3 = lin#
    return l3

In [ ]:
%timeit -n 10 _=model(x_valid)

8.04 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [ ]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [ ]:
#export
def mse(output, targ): return #

In [ ]:
y_train,y_valid = #

In [ ]:
preds = #

In [ ]:
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(31.2716)

### Gradients and backward pass

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    #

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    #

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    #
    #
    #

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = #
    l2 = relu(l1)
    out = #
    # we don't actually need the loss in backward!
    loss = #
    
    # backward pass:
    #
    #
    #
    #

In [ ]:
forward_and_backward(x_train, y_train)

In [ ]:
# Save for testing against later
w1g = #
w2g = #
b1g = #
b2g = #
ig  = x_train.#

We cheat a little bit and use PyTorch autograd to check our results.

In [ ]:
xt2 = x_train.clone()#
w12 = w1.clone()#
w22 = w2.clone()#
b12 = b1.clone()#
b22 = b2.clone()#

In [ ]:
def forward(inp, targ):
    # forward pass:
    l1 = #
    l2 = #
    out = #
    # we don't actually need the loss in backward!
    return #

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
#

In [ ]:
test_near(##, w2g)
test_near(##, b2g)
test_near(w12.grad, ##)
test_near(##, b1g)
test_near(xt2.grad, ##)

## Refactor model

### Layers as classes

In [ ]:
class Relu():
    def __call__#
#
#
#
    
    def backward#

In [ ]:
class Lin():
    def __init__#
        
    def __call__#
#
#
#
    
    def backward#
#
        # Creating a giant outer product, just to sum it, is inefficient!
#
#

In [ ]:
class Mse():
    def __call__#
#
#
#
#
    
    def backward#
#

In [ ]:
class Model():
    def __init__#
#
#
        
    def __call__#
#
#
    
    def backward#
#
#

In [ ]:
w1.g,#
model = #

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 144 ms, sys: 7.49 ms, total: 151 ms
Wall time: 76 ms


In [ ]:
%time model.backward()

CPU times: user 4.66 s, sys: 4.85 s, total: 9.51 s
Wall time: 5.13 s


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [ ]:
class Module():
    def __call__#
#
#
#
    
    def forward#
    def backward#

In [ ]:
class Relu#
    def forward#
    def bwd#

In [ ]:
class Lin#
    def __init__#
        
    def forward#
    
    def bwd#
        #
        #
        #

In [ ]:
class Mse#
    def forward#
    def bwd#

In [ ]:
class Model():
    def __init__#
        #
        #
        
    def __call__#
        #
        #
    
    def backward#
        #
        #

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 144 ms, sys: 9.14 ms, total: 153 ms
Wall time: 83.5 ms


In [ ]:
%time model.backward()

CPU times: user 311 ms, sys: 178 ms, total: 488 ms
Wall time: 286 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [ ]:
class Lin#
    def __init__#
        
    def forward#
    
    def bwd#
        #
        #
        #

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 159 ms, sys: 5.16 ms, total: 164 ms
Wall time: 184 ms


In [ ]:
%time model.backward()

CPU times: user 311 ms, sys: 140 ms, total: 451 ms
Wall time: 450 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [ ]:
#export
from ## import nn

In [ ]:
class Model#
    def __init__#
        #super().__init__()
        #
        #
        
    def __call__#
        #
        #

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 152 ms, sys: 5.31 ms, total: 157 ms
Wall time: 222 ms


In [ ]:
%time loss.backward()

CPU times: user 169 ms, sys: 39.7 ms, total: 209 ms
Wall time: 243 ms


## Export

In [ ]:
!python notebook2script.py 02_fully_connected.ipynb